<a href="https://colab.research.google.com/github/kmeagle1515/VAE/blob/master/variational_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler
from keras.datasets import mnist
import keras

import numpy as np
import keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt

m=50
n_z=2
n_epoch=10

#encoder 
inputs=Input(shape=(784,))
h_q=Dense(512, activation='relu')(inputs)   #why 512 ?
mu=Dense(n_z, activation='linear')(h_q)
log_sig=Dense(n_z, activation='linear')(h_q)

#sampling latent variable
def sample_z(args):
    mu,log_sig= args
    eps=K.random_normal(shape=(m,n_z), mean= 0 , stddev= 1)
    return mu + K.exp(log_sig / 2)*eps

#z ~Q(Z/X)
z= Lambda(sample_z)([mu, log_sig])

#decoder

dec_hid= Dense(512, activation='relu')
dec_out= Dense(784, activation= 'sigmoid')

h_p=dec_hid(z)
out=dec_out(h_p)
d_in=Input(shape=(n_z, ))
d_h= dec_hid(d_in)
d_ou=dec_out(d_h)

vae=Model(inputs,out)
encoder=Model(inputs, mu)


def vae_loss(y_true, y_pred):
    """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
    # E[log P(X|z)]
    recon = K.sum(K.binary_crossentropy(y_pred, y_true), axis=1)
    # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
    kl = 0.5 * K.sum(K.exp(log_sig) + K.square(mu) - 1. - log_sig, axis=1)

    return recon + kl

vae.compile(optimizer='adam', loss=vae_loss)


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

vae.fit(x_train, x_train, batch_size=m, nb_epoch=n_epoch)



Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


Epoch 1/10
60000/60000 [==============================] - 26s 428us/step - loss: 491.6520
Epoch 2/10
60000/60000 [==============================] - 24s 408us/step - loss: 432.7177
Epoch 3/10
60000/60000 [==============================] - 25s 416us/step - loss: 431.2271
Epoch 4/10
60000/60000 [==============================] - 24s 408us/step - loss: 431.0615
Epoch 5/10
60000/60000 [==============================] - 24s 402us/step - loss: 431.0524
Epoch 6/10
60000/60000 [==============================] - 24s 395us/step - loss: 431.0502
Epoch 7/10
60000/60000 [==============================] - 25s 419us/step - loss: 431.0494
Epoch 8/10
60000/60000 [==============================] - 25s 418us/step - loss: 431.0489
Epoch 9/10
60000/60000 [==============================] - 25s 414us/step - loss: 431.0486
Epoch 10/10
60000/60000 [==============================] - 25s 423us/step - loss: 431.0484
